In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, Dropout, MaxPooling1D, Input
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import pickle

In [2]:
# Load preprocessed data
preprocessed_data = pd.read_pickle("C:/Users/syafi/Desktop/syafiq-project/classification-task/model/saved_data/preprocessed_data.pkl")
X_scaled = preprocessed_data["X_scaled"]
y_resampled = preprocessed_data["y_resampled"]
class_labels = preprocessed_data["class_labels"]

original_features = pd.read_pickle("C:/Users/syafi/Desktop/syafiq-project/classification-task/model/saved_data/original_features.pkl")

In [3]:
# Sample sizes for training
sample_sizes = [25, 50, 75, 100, 250, 500, 750, 1000, 2500, 5000, 7500, 10000, 20000, 30000, 40000, 50000]

In [4]:
# Function to create a stratified subsample
def stratified_subsample(X, y, train_size, random_state=42):
    if len(X) < train_size:
        raise ValueError(f"Insufficient data for sample size {train_size}.")
    X_sample, _, y_sample, _ = train_test_split(
        X, y, train_size=train_size, stratify=y, random_state=random_state
    )
    return X_sample, y_sample

In [5]:
def build_cnn(input_shape, num_classes):
    model = Sequential(
        [
            Input(shape=input_shape),
            Conv1D(filters=32, kernel_size=3, activation="relu"),
            MaxPooling1D(pool_size=2),
            Dropout(0.25),
            Conv1D(filters=64, kernel_size=3, activation="relu"),
            MaxPooling1D(pool_size=2),
            Dropout(0.25),
            Flatten(),
            Dense(128, activation="relu"),
            Dropout(0.5),
            Dense(num_classes, activation="softmax"),
        ]
    )
    model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model

In [7]:
# Initialize and train CNN with different sample sizes
results = {}

for sample_size in sample_sizes:
    try:
        X_sample, y_sample = stratified_subsample(X_scaled, y_resampled, sample_size)

        test_size = max(0.2, len(class_labels) / len(y_sample))
        stratify = y_sample if len(y_sample) >= len(class_labels) else None

        X_train, X_test, y_train, y_test = train_test_split(
            X_sample, y_sample, test_size=test_size, stratify=stratify, random_state=42
        )

        # Reshape data for CNN
        input_shape = (X_train.shape[1], 1)
        X_train = X_train.to_numpy().reshape(-1, X_train.shape[1], 1)
        X_test = X_test.to_numpy().reshape(-1, X_test.shape[1], 1)

        y_train = to_categorical(y_train, num_classes=len(class_labels))
        y_test_labels = y_test  # Save original labels for evaluation
        y_test = to_categorical(y_test, num_classes=len(class_labels))

        # Build and train CNN model
        cnn_model = build_cnn(input_shape=input_shape, num_classes=len(class_labels))
        cnn_model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=2)

        # Evaluate the model
        y_pred = cnn_model.predict(X_test)
        y_pred_classes = np.argmax(y_pred, axis=1)
        accuracy = accuracy_score(y_test_labels, y_pred_classes)
        print(f"Classification Report for Sample Size {sample_size}:")
        print(
            classification_report(
                y_test_labels, y_pred_classes, target_names=class_labels
            )
        )

        results[sample_size] = {
            "accuracy": accuracy,
            "classification_report": classification_report(
                y_test_labels,
                y_pred_classes,
                target_names=class_labels,
                output_dict=True,
                zero_division=True
            ),
        }

    except ValueError as e:
        print(f"Skipping sample size {sample_size}: {e}")

Epoch 1/10
1/1 - 2s - 2s/step - accuracy: 0.0714 - loss: 76.4326
Epoch 2/10
1/1 - 0s - 39ms/step - accuracy: 0.1429 - loss: 61.2299
Epoch 3/10
1/1 - 0s - 47ms/step - accuracy: 0.0714 - loss: 55.0157
Epoch 4/10
1/1 - 0s - 37ms/step - accuracy: 0.0000e+00 - loss: 50.6652
Epoch 5/10
1/1 - 0s - 37ms/step - accuracy: 0.2857 - loss: 40.6066
Epoch 6/10
1/1 - 0s - 34ms/step - accuracy: 0.1429 - loss: 34.4358
Epoch 7/10
1/1 - 0s - 37ms/step - accuracy: 0.1429 - loss: 57.8613
Epoch 8/10
1/1 - 0s - 38ms/step - accuracy: 0.1429 - loss: 51.5191
Epoch 9/10
1/1 - 0s - 35ms/step - accuracy: 0.0714 - loss: 49.9975
Epoch 10/10
1/1 - 0s - 32ms/step - accuracy: 0.2857 - loss: 57.6435
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
Classification Report for Sample Size 25:
                                      precision    recall  f1-score   support

                             Anaemia       0.14      1.00      0.25         1
                          Arrhythmia       0.00      0.00      0.00         1
           

c:\Users\syafi\anaconda3\envs\mydatascienv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\syafi\anaconda3\envs\mydatascienv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\syafi\anaconda3\envs\mydatascienv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

Epoch 1/10
2/2 - 2s - 782ms/step - accuracy: 0.1026 - loss: 89.6331
Epoch 2/10
2/2 - 0s - 19ms/step - accuracy: 0.0513 - loss: 61.6747
Epoch 3/10
2/2 - 0s - 20ms/step - accuracy: 0.1282 - loss: 57.3294
Epoch 4/10
2/2 - 0s - 22ms/step - accuracy: 0.0769 - loss: 59.8381
Epoch 5/10
2/2 - 0s - 20ms/step - accuracy: 0.1026 - loss: 53.8653
Epoch 6/10
2/2 - 0s - 16ms/step - accuracy: 0.1282 - loss: 40.8117
Epoch 7/10
2/2 - 0s - 23ms/step - accuracy: 0.1026 - loss: 39.0653
Epoch 8/10
2/2 - 0s - 22ms/step - accuracy: 0.0256 - loss: 44.0798
Epoch 9/10
2/2 - 0s - 22ms/step - accuracy: 0.0769 - loss: 41.6643
Epoch 10/10
2/2 - 0s - 19ms/step - accuracy: 0.0769 - loss: 40.7067
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
Classification Report for Sample Size 50:
                                      precision    recall  f1-score   support

                             Anaemia       0.00      0.00      0.00         1
                          Arrhythmia       0.00      0.00      0.00         1
            

c:\Users\syafi\anaconda3\envs\mydatascienv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\syafi\anaconda3\envs\mydatascienv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\syafi\anaconda3\envs\mydatascienv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

Epoch 1/10
2/2 - 1s - 742ms/step - accuracy: 0.0833 - loss: 69.0972
Epoch 2/10
2/2 - 0s - 22ms/step - accuracy: 0.1000 - loss: 66.3576
Epoch 3/10
2/2 - 0s - 24ms/step - accuracy: 0.1000 - loss: 47.2214
Epoch 4/10
2/2 - 0s - 17ms/step - accuracy: 0.0333 - loss: 49.0360
Epoch 5/10
2/2 - 0s - 23ms/step - accuracy: 0.1667 - loss: 40.2576
Epoch 6/10
2/2 - 0s - 17ms/step - accuracy: 0.0833 - loss: 40.0107
Epoch 7/10
2/2 - 0s - 34ms/step - accuracy: 0.1333 - loss: 37.0125
Epoch 8/10
2/2 - 0s - 22ms/step - accuracy: 0.0833 - loss: 35.4826
Epoch 9/10
2/2 - 0s - 19ms/step - accuracy: 0.1000 - loss: 28.6977
Epoch 10/10
2/2 - 0s - 23ms/step - accuracy: 0.1500 - loss: 24.5692
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
Classification Report for Sample Size 75:
                                      precision    recall  f1-score   support

                             Anaemia       0.07      1.00      0.12         1
                          Arrhythmia       0.00      0.00      0.00         1
             

...

c:\Users\syafi\anaconda3\envs\mydatascienv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\syafi\anaconda3\envs\mydatascienv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\syafi\anaconda3\envs\mydatascienv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

Epoch 1/10
500/500 - 3s - 7ms/step - accuracy: 0.0944 - loss: 6.0381
Epoch 2/10
500/500 - 2s - 3ms/step - accuracy: 0.0897 - loss: 2.4058
Epoch 3/10
500/500 - 2s - 4ms/step - accuracy: 0.0886 - loss: 2.4016
Epoch 4/10
500/500 - 2s - 4ms/step - accuracy: 0.0872 - loss: 2.4011
Epoch 5/10
500/500 - 2s - 4ms/step - accuracy: 0.0892 - loss: 2.3989
Epoch 6/10
500/500 - 2s - 4ms/step - accuracy: 0.0873 - loss: 2.3990
Epoch 7/10
500/500 - 2s - 4ms/step - accuracy: 0.0869 - loss: 2.3985
Epoch 8/10
500/500 - 2s - 4ms/step - accuracy: 0.0878 - loss: 2.3988
Epoch 9/10
500/500 - 2s - 4ms/step - accuracy: 0.0857 - loss: 2.3981
Epoch 10/10
500/500 - 2s - 4ms/step - accuracy: 0.0886 - loss: 2.3988
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Classification Report for Sample Size 20000:
                                      precision    recall  f1-score   support

                             Anaemia       0.00      0.00      0.00       364
                          Arrhythmia       0.00      0.00      0.0

c:\Users\syafi\anaconda3\envs\mydatascienv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\syafi\anaconda3\envs\mydatascienv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\syafi\anaconda3\envs\mydatascienv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

Epoch 1/10
750/750 - 5s - 7ms/step - accuracy: 0.0940 - loss: 4.8030
Epoch 2/10
750/750 - 3s - 4ms/step - accuracy: 0.0924 - loss: 2.4005
Epoch 3/10
750/750 - 3s - 4ms/step - accuracy: 0.1188 - loss: 2.3338
Epoch 4/10
750/750 - 3s - 4ms/step - accuracy: 0.1262 - loss: 2.2780
Epoch 5/10
750/750 - 3s - 4ms/step - accuracy: 0.1266 - loss: 2.2681
Epoch 6/10
750/750 - 3s - 4ms/step - accuracy: 0.1479 - loss: 2.2375
Epoch 7/10
750/750 - 3s - 4ms/step - accuracy: 0.1775 - loss: 2.1870
Epoch 8/10
750/750 - 3s - 4ms/step - accuracy: 0.2107 - loss: 2.1063
Epoch 9/10
750/750 - 3s - 3ms/step - accuracy: 0.2424 - loss: 2.0382
Epoch 10/10
750/750 - 3s - 4ms/step - accuracy: 0.2897 - loss: 1.9264
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Classification Report for Sample Size 30000:
                                      precision    recall  f1-score   support

                             Anaemia       0.29      0.05      0.08       546
                          Arrhythmia       0.23      0.81      0.3

c:\Users\syafi\anaconda3\envs\mydatascienv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\syafi\anaconda3\envs\mydatascienv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\syafi\anaconda3\envs\mydatascienv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

In [ ]:
# Save the best model (assuming last one is the best for simplicity)
cnn_model.save("cnn_model.h5")
print("CNN model saved as cnn_model.h5")

In [ ]:
# Load and visualize results
for sample_size, metrics in results.items():
    print(f"Sample Size: {sample_size} - Accuracy: {metrics['accuracy']:.4f}")